In [1]:
!pip install selenium

  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 53.1 MB/s eta 0:00:00
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib

In [11]:
!pip install undetected-chromedriver

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'undetected-chromedriver' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'undetected-chromedriver'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47129 sha256=9fbc698c1e4a9a617bc0c96fff93ee1a70b5ed289e3f78a00c6d06987418f3f0
  Stored in directory: /Users/kwoneunei/Library/Caches/pip/wheels/cf/a1/db/e1275b6f7259aacd6b045f8bfcb1fcbc93827a3916ba55d5b7
Successfully built undetected-chromedriver
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [undetected-chromedriver]


In [13]:
!pip install -U undetected-chromedriver

In [30]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 43.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [playwright]━━━━━━━ 2/3 [playwright]


In [33]:
!pip install playwright nest_asyncio

In [61]:
import os, tempfile, time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

USER = "kwoneunei"
PW   = "kwoneunei0511@@"

# 실제 Chrome 실행파일 경로 (맥)
CHROME_BIN = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"

# 충돌 피하려고 임시 프로필 디렉터리 사용
USER_DATA_DIR = os.path.join(tempfile.gettempdir(), "uc_profile_everytime")
os.makedirs(USER_DATA_DIR, exist_ok=True)

opts = uc.ChromeOptions()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_argument(f"--user-data-dir={USER_DATA_DIR}")   # 새 프로필
opts.add_argument("--profile-directory=Default")        # 새 프로필 안의 기본 프로필

# 핵심: 실제 Chrome 경로를 명시, 버전은 140
driver = uc.Chrome(
    options=opts,
    browser_executable_path=CHROME_BIN,  # ← 실제 크롬 바이너리
    version_main=140,
    use_subprocess=True                  # 맥에서 안정성 ↑
)

wait = WebDriverWait(driver, 15)

driver.get("https://everytime.kr/login")

# 로그인 폼 채우기 (name="id", "password")
wait.until(EC.visibility_of_element_located((By.NAME, "id"))).send_keys(USER)
driver.find_element(By.NAME, "password").send_keys(PW)

# 제출 (버튼 or form submit)
btns = driver.find_elements(By.XPATH, '//input[@type="submit" and contains(@value,"로그인")]')
(btns[0].click() if btns else driver.execute_script(
    'document.querySelector(\'form[action="/user/login"]\').submit();'
))

# 결과 대기
try:
    WebDriverWait(driver, 12).until(EC.any_of(
        EC.url_contains("everytime.kr"),
        EC.presence_of_element_located((By.XPATH, '//a[contains(.,"게시판")]'))
    ))
    print("로그인 성공(추정):", driver.title)
except Exception as e:
    print("로그인 실패/차단:", e)
    driver.save_screenshot("everytime_after_login.png")

로그인 성공(추정): 로그인 - 에브리타임


In [62]:
# -------- 로그인 이후부터 붙여 실행 --------
import re, csv, time, urllib.parse
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

wait = WebDriverWait(driver, 15)

COURSE_QUERY = "시스템프로그래밍"
PROFESSOR    = "이은지"
MAX_REVIEWS  = 100

def save_debug(prefix="debug"):
    try: driver.save_screenshot(f"{prefix}.png")
    except: pass
    try:
        with open(f"{prefix}.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)
    except: pass
    print(f"[디버그 저장] {prefix}.png, {prefix}.html")

def first_visible(cands, timeout=12):
    w = WebDriverWait(driver, timeout)
    last = None
    for by, sel in cands:
        try:
            return w.until(EC.visibility_of_element_located((by, sel)))
        except Exception as e:
            last = e
            continue
    if last: raise last
    raise TimeoutException(f"요소를 찾지 못했습니다: {cands}")

# ── 1) 교수명 검색 → 과목 카드 클릭
def open_course_by_prof(course, prof):
    prof_q = urllib.parse.quote(prof)
    driver.get(f"https://everytime.kr/lecture/search?keyword={prof_q}&condition=professor")
    time.sleep(1.2)

    seen = set()
    for _ in range(20):
        cards = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/lecture/view/"]')
        target = None
        for a in cards:
            try:
                href = a.get_attribute("href") or ""
                if href in seen: continue
                text = (a.text or a.get_attribute("innerText") or "").strip()
                norm = " ".join(text.split())
                if course in norm and prof in norm:
                    target = a
                    break
                seen.add(href)
            except:
                continue
        if target:
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", target)
            time.sleep(0.3)
            target.click()
            time.sleep(1.5)
            print(f"[OK] '{prof}' 교수 '{course}' 강의실 진입 시도")
            return True
        driver.execute_script("window.scrollBy(0, Math.max(600, window.innerHeight*0.9));")
        time.sleep(0.8)

    print("[에러] 교수/과목 조합을 못 찾음")
    save_debug("prof_search_no_match")
    return False

# ── 2) 현재 페이지가 맞는 교수/과목인지 확인 (아니면 1단계 재시도)
def ensure_correct_prof(course, prof):
    body = driver.find_element(By.TAG_NAME, "body").text
    if (course in body) and (prof in body):
        return True
    # 상세 페이지 내 교수 링크 경유 재탐색
    try:
        prof_links = driver.find_elements(By.XPATH, f'//a[contains(@href,"/lecture/search") and contains(.,"{prof}")]')
        if prof_links:
            prof_links[0].click(); time.sleep(1.0)
            return open_course_by_prof(course, prof)
    except:
        pass
    return open_course_by_prof(course, prof)

# ── 3) 강의평 탭 진입 (href 추출 → 절대 URL 이동 → JS 클릭 백업 → 다형 대기)
def switch_to_review_tab():
    base = "https://everytime.kr"
    # 3-1) 탭바에서 '강의평' a의 href를 얻어 절대 URL로 이동
    try:
        tab_a = first_visible([
            (By.XPATH, '//div[contains(@class,"tabbar")]//a[contains(@href,"tab=article")]'),
            (By.XPATH, '//div[contains(@class,"tabbar")]//span[text()="강의평"]/ancestor::a[1]'),
            (By.CSS_SELECTOR, 'a[href*="?tab=article"]')
        ], timeout=6)
        href = tab_a.get_attribute("href") or tab_a.get_attribute("data-href") or ""
        if href:
            abs_url = href if href.startswith("http") else urljoin(base, href)
            driver.get(abs_url)
            time.sleep(0.8)
    except Exception:
        pass

    # 3-2) 여전히 탭 전환이 안 되면 DOM 클릭(오버레이 대비 스크롤 후 클릭)
    try:
        tab_a = first_visible([
            (By.XPATH, '//div[contains(@class,"tabbar")]//span[text()="강의평"]/ancestor::a[1]'),
            (By.XPATH, '//a[contains(@href,"tab=article")]')
        ], timeout=4)
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", tab_a)
        time.sleep(0.2)
        driver.execute_script("arguments[0].click();", tab_a)
        time.sleep(0.8)
    except Exception:
        pass

    # 3-3) 다양한 구조로 로드 대기 (SPA/빌드별 다형성)
    try:
        WebDriverWait(driver, 12).until(
            EC.any_of(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.review div.articles div.article')),
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.articles div.article')),
                EC.presence_of_element_located((By.XPATH, '//div[contains(@class,"review")]//div[contains(@class,"articles")]//div[contains(@class,"article")]')),
            )
        )
        print("[OK] 강의평 탭 진입")
    except TimeoutException as e:
        print("[에러] 강의평 탭 진입 실패(대기 초과):", e)
        save_debug("switch_review_tab_fail")
        # 마지막 백업: location.search 강제
        try:
            driver.execute_script('if(location.search.indexOf("tab=article")===-1){location.search="?tab=article";}')
            time.sleep(1.0)
            WebDriverWait(driver, 8).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.articles div.article, div.review .articles .article'))
            )
            print("[OK] 강제 전환으로 진입")
        except Exception:
            save_debug("switch_review_tab_fail_after_force")
            raise

# ── 4) 강의평 크롤링
def load_more_if_any():
    for _ in range(4):
        btns = driver.find_elements(
            By.XPATH,
            '//a[contains(@class,"more") and contains(.,"강의평")] | //button[contains(.,"더보기") or contains(.,"더 보기")]'
        )
        if not btns: break
        try:
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btns[0])
            time.sleep(0.2)
            btns[0].click()
            time.sleep(1.0)
        except Exception:
            break

def scrape_reviews(max_n=100):
    reviews, seen = [], set()
    last_h = 0
    for _ in range(15):
        blocks = driver.find_elements(By.CSS_SELECTOR, "div.review div.articles div.article")
        if not blocks:
            blocks = driver.find_elements(By.CSS_SELECTOR, "div.articles div.article")
        if not blocks:
            blocks = driver.find_elements(By.XPATH, '//div[contains(@class,"article") and .//div[contains(@class,"text")]]')

        for b in blocks:
            try:
                # 본문
                try:
                    text_el = b.find_element(By.CSS_SELECTOR, "div.text")
                    text = text_el.text.strip()
                except NoSuchElementException:
                    text = b.text.strip()
                if not text: 
                    continue
                norm = re.sub(r"\s+", " ", text)
                if norm in seen: 
                    continue
                seen.add(norm)

                # 별점 (폭 % → 5점 환산)
                star = ""
                try:
                    star_el = b.find_element(By.CSS_SELECTOR, "span.star span.on")
                    style = star_el.get_attribute("style") or ""
                    m = re.search(r'width:\s*([\d.]+)%', style)
                    if m:
                        pct = float(m.group(1))
                        star = f"{round(pct/20, 2):.2f}"
                except NoSuchElementException:
                    pass

                # 학기
                semester = ""
                try:
                    sem_el = b.find_element(By.CSS_SELECTOR, "span.semester")
                    semester = sem_el.text.strip()
                except NoSuchElementException:
                    pass

                reviews.append({"star": star, "semester": semester, "text": norm})
                if len(reviews) >= max_n:
                    return reviews
            except Exception:
                continue

        load_more_if_any()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.0)
        h = driver.execute_script("return document.body.scrollHeight")
        if h == last_h: break
        last_h = h
    return reviews

# ── 실행 플로우
if not ensure_correct_prof(COURSE_QUERY, PROFESSOR):
    raise SystemExit

switch_to_review_tab()
time.sleep(0.6)  # 렌더링 안정화

data = scrape_reviews(MAX_REVIEWS)
print(f"[수집] 강의평 {len(data)}개")

safe = re.sub(r"[^\w가-힣]+", "_", f"{COURSE_QUERY}_{PROFESSOR}")
csv_path = f"reviews_{safe}.csv"
with open(csv_path, "w", newline="", encoding="utf-8-sig") as f:
    w = csv.DictWriter(f, fieldnames=["star","semester","text"])
    w.writeheader()
    w.writerows(data)
print("CSV 저장:", csv_path)

save_debug(f"reviews_page_{safe}")

[OK] '이은지' 교수 '시스템프로그래밍' 강의실 진입 시도
[OK] 강의평 탭 진입
[수집] 강의평 5개
CSV 저장: reviews_시스템프로그래밍_이은지.csv
[디버그 저장] reviews_page_시스템프로그래밍_이은지.png, reviews_page_시스템프로그래밍_이은지.html
